In [1]:
#Using InsideAirbnb's dataset I analyzed Amsterdam's Airbnb data.
import pandas as pd
import numpy as np

# Loading 'id','price','neighbourhood' columns to to the variable prices and printing the resulting dataframe
prices = pd.read_csv(r"Amsterdam_airbnb_listings_05_June_2022.csv", usecols=['id','price','neighbourhood'])
print(prices)

# Loading 'id','room_type','name' columns to to the variable room_types and printing the resulting dataframe
room_types = pd.read_csv(r"Amsterdam_airbnb_listings_05_June_2022.csv", usecols=['id','room_type','name'])
print(room_types)

# Loading 'id','host_name','last_review'columns to to the variable reviews and printing the resulting dataframe
reviews = pd.read_csv(r"Amsterdam_airbnb_listings_05_June_2022.csv", usecols=['id','host_name','last_review'])
print(reviews)

            id             neighbourhood  price
0     23726706  IJburg - Zeeburgereiland     88
1     35815036                Noord-Oost    105
2     31553121                Noord-West    152
3     34745823     Gaasperdam - Driemond     87
4     44586947     Gaasperdam - Driemond    160
...        ...                       ...    ...
6168  48005583    Buitenveldert - Zuidas    142
6169  29532926           Watergraafsmeer     95
6170  35760705                Noord-West    180
6171  36900951    Buitenveldert - Zuidas    174
6172  40575103           Watergraafsmeer     65

[6173 rows x 3 columns]
            id                                               name  \
0     23726706  Private room 20 minutes from Amsterdam + Break...   
1     35815036  Vrijstaand vakantiehuis, privé tuin aan het water   
2     31553121                    Quiet Guesthouse near Amsterdam   
3     34745823           Apartment ' Landzicht', nearby Amsterdam   
4     44586947                  Weesp, 2 kamers vlakbi

In [2]:
# Converting the prices column to numeric datatype
prices["price"] = pd.to_numeric(prices["price"])

# Printing descriptive statistics for the price column
print(prices["price"].describe())

count    6173.000000
mean      198.019601
std       140.546979
min         0.000000
25%       115.000000
50%       160.000000
75%       240.000000
max      2500.000000
Name: price, dtype: float64


In [3]:
# Checking if there are any subset prices for listings costing $0 and saving it to the free_listings variable
free_listings = prices["price"] == 0

# Updating prices by removing all free listings from prices if there are any
prices = prices.loc[~free_listings]

# Calculating the average price and rounding it for 2 nearest decimals and saving it to the variable avg_price 
avg_price = round(prices["price"].mean(), 2)

# Printing a message with the calculated average price per night
print("The average price per night for an Airbnb listing in Amsterdam is ${}.".format(avg_price))

The average price per night for an Airbnb listing in Amsterdam is $198.28.


In [4]:
# Adding a new column called price_per_month to the prices dataframe 
prices["price_per_month"] = prices["price"] * 90 / 3

# Calculating average_price_per_month and rounding it to the 2 nearest decimals
average_price_per_month = round(prices["price_per_month"].mean(), 2)

# Printing the average_price_per_month in Amsterdam
print("Airbnb monthly costs in Amsterdam are ${}.".format(average_price_per_month))

Airbnb monthly costs in Amsterdam are $5948.3.


In [5]:
# Converting the room_type column to lowercase
room_types["room_type"] = room_types["room_type"].str.lower()

# Updating the room_type column to category data type
room_types["room_type"] = room_types["room_type"].astype("category")

# Saving the frequencie of each type of room to the variable room_frequencies
room_frequencies = room_types["room_type"].value_counts()

# Print room_frequencies to all types of room on Airbnb
print(room_frequencies)

entire home/apt    4223
private room       1840
hotel room           78
shared room          32
Name: room_type, dtype: int64


In [6]:
# Merging prices and room_types to create the variable rooms_and_prices
rooms_and_prices = prices.merge(room_types, how="outer", on="id")

# Merging rooms_and_prices with the reviews DataFrame to create the variable airbnb_merged
airbnb_merged = rooms_and_prices.merge(reviews, how="outer", on="id")

# Droping any missing values from airbnb_merged to leave a clean dataframe
airbnb_merged = airbnb_merged.dropna()

# Checking if there are any duplicate values on airbnb_merged
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))
print(rooms_and_prices)

There are 0 duplicates in the DataFrame.
            id             neighbourhood  price  price_per_month  \
0     23726706  IJburg - Zeeburgereiland   88.0           2640.0   
1     35815036                Noord-Oost  105.0           3150.0   
2     31553121                Noord-West  152.0           4560.0   
3     34745823     Gaasperdam - Driemond   87.0           2610.0   
4     44586947     Gaasperdam - Driemond  160.0           4800.0   
...        ...                       ...    ...              ...   
6168  43095925                       NaN    NaN              NaN   
6169  43148414                       NaN    NaN              NaN   
6170  46273178                       NaN    NaN              NaN   
6171  46273734                       NaN    NaN              NaN   
6172  47110768                       NaN    NaN              NaN   

                                                   name        room_type  
0     Private room 20 minutes from Amsterdam + Break...     private

In [7]:
# Extracting information from the neighbourhood column and store as a new column n_hood
airbnb_merged["n_hood"] = airbnb_merged["neighbourhood"].str.partition(",")[0]

# Grouping by neighbourhood and calculate summary statistics
neighbourhoods = airbnb_merged.groupby("n_hood")["price"].agg(["sum", "mean", "median", "count"])

# Rounding neighbourhoods to 2 nearest decimal places and sort by mean in descending order
neighbourhoods = neighbourhoods.round(2).sort_values("mean", ascending=False)

# Printing neighbourhoods
print(neighbourhoods)

                                             sum    mean  median  count
n_hood                                                                 
Centrum-Oost                            127426.0  233.38   185.0    546
Centrum-West                            179842.0  228.23   185.0    788
De Pijp - Rivierenbuurt                 110733.0  206.98   175.0    535
Zuid                                     68209.0  206.69   177.0    330
Oud-Noord                                43737.0  197.90   150.0    221
Westerpark                               74697.0  192.02   160.0    389
De Baarsjes - Oud-West                  153655.0  191.11   155.0    804
Buitenveldert - Zuidas                   19618.0  188.63   162.0    104
IJburg - Zeeburgereiland                 27579.0  180.25   150.0    153
Oud-Oost                                 53530.0  179.03   155.0    299
Watergraafsmeer                          32630.0  177.34   150.0    184
Geuzenveld - Slotermeer                  16596.0  176.55   137.0

In [8]:
# Create labels for the price range and saving as variable label_names
label_names = ["Budget", "Average", "Expensive", "Extravagant"]

# Create the label ranges and saving as variable ranges
ranges = [0, 75, 150, 300, np.inf]

# Inserting new column price_range into DataFrame
airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)

# Calculate occurence frequencies for each label, prices_by_neighbourhood
prices_by_neighbourhood = airbnb_merged.groupby(['neighbourhood', 'price_range'])['price_range'].count()
print(prices_by_neighbourhood)

neighbourhood    price_range
Bijlmer-Centrum  Budget          16
                 Average         33
                 Expensive       20
                 Extravagant      1
Bijlmer-Oost     Budget           7
                               ... 
Westerpark       Extravagant     46
Zuid             Budget          12
                 Average        130
                 Expensive      131
                 Extravagant     57
Name: price_range, Length: 88, dtype: int64
